# SETUP

# TODO
- Aggiungere cella per diminuire calcoli (e dimensione dataset?)
- Creare un RDD i cui elementi siano le coppie righe/colonne e con un'operazione di map/reduce emettere coppie chiave-valore nelle quali la chiave indica la posizione (i, j) dell'elemento della matrice da aggiornare e il valore indica il nuovo elemento della matrice, utilizzando le formule del paragrafo 9.4.4. In questo modo "parallelizzate" gli aggiornamenti di tutti gli elementi della matrice. Eventualmente potete evitare di aggiornare d'un colpo tutta la matrice (potrebbe portare a difficoltà nella convergenza), decidendo in modo probabilistico ogni volta se aggiornare oppure no un elemento: basta che il job map/reduce generi un bit pseudocasuale che vale 1 con probabilità p, e se il suo valore è 1 allora utilizzate le formule di cui sopra, altrimenti restituite il valore attuale di U (fa parte della colonna che ottenete in input). In questo modo aggiornerete all'incirca una frazione pari a p dei valori della matrice. Lo stesso vale per V.

In [13]:
if "google.colab" in str(get_ipython()):
    !pip install pyspark
    !git clone https://github.com/lukebella/YelpRecommenderSystem.git
    !mv YelpRecommenderSystem/* .
    !rm -fr YelpRecommenderSystem


In [14]:
import os

os.environ['KAGGLE_USERNAME'] = "xxxxxx"
os.environ['KAGGLE_KEY'] = "xxxxxx"
!kaggle datasets download -p ./data -d yelp-dataset/yelp-dataset
!unzip -n ./data/yelp-dataset.zip -d ./data


zsh:1: command not found: kaggle
unzip:  cannot find or open ./data/yelp-dataset.zip, ./data/yelp-dataset.zip.zip or ./data/yelp-dataset.zip.ZIP.


In [15]:
review_filename = 'data/yelp_academic_dataset_review.json'
user_filename = 'data/yelp_academic_dataset_user.json'
business_filename = 'data/yelp_academic_dataset_business.json'

# Lavorare su un sample più piccolo (10%)
### conviente farlo solo sulle review per problemi di join

In [16]:
from pyspark.sql import SparkSession

# .master().config('spark.driver.memory', "15g")
spark = SparkSession.builder.getOrCreate()
# sc = pyspark.SparkContext().getOrCreate()
sc = spark.sparkContext


In [17]:
# TODO add docstring
def from_json_to_RDD(filename):
    raw_df = spark.read.json(filename)
    raw_df.show()
    #return raw_df.sample(frac=1/10, random_state = 0).dropna().rdd
    return raw_df.rdd

raw_review_RDD = from_json_to_RDD(review_filename).sample(True, 1, 0)  #699192 rows
raw_user_RDD = from_json_to_RDD(user_filename)
raw_business_RDD = from_json_to_RDD(business_filename)
#print(raw_business_RDD.filter(lambda x: x[2]=='bqFG0QJY9jj2m55OqAVHeA').first())

raw_review_RDD.first()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+---------+------------+------+--------------------+-------------------+
|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer| cool|               elite|fans|             friends|funny|     name|review_count|useful|             user_id|      yelping_since|
+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+---------+------------+------+--------------------+-------------------+
|         

Row(business_id='XQfwVwDr-v0ZS3_CbbE5Xw', cool=0, date='2018-07-07 22:09:11', funny=0, review_id='KU_O5udG6zpxOg-VcAEodg', stars=3.0, text="If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.", useful=0, user_id='mh_-eMZ6K5RLWhZyISBhwA')

In [18]:
raw_user_RDD = raw_user_RDD.sample(True, 1/600, 0)
user_list = raw_user_RDD.map(lambda x: x[-2]).collect()

raw_review_RDD = raw_review_RDD.filter(lambda x: (x[8] in user_list))
print(len(user_list))
raw_review_RDD.count()

3292


Exception in thread "serve RDD 52 with partitions 0" java.net.SocketTimeoutException: Accept timed out
	at java.base/sun.nio.ch.NioSocketImpl.timedAccept(NioSocketImpl.java:694)
	at java.base/sun.nio.ch.NioSocketImpl.accept(NioSocketImpl.java:738)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:690)
	at java.base/java.net.ServerSocket.platformImplAccept(ServerSocket.java:655)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:631)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:588)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:546)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)


12205

# RETRIEVE RDDs

In [19]:
def get_review_tuple(entry):
    """ Parse a row in the review dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the review dataset in JSON format
    Returns:
        tuple: (review_id, user_id, business_id, stars, useful, funny, cool, text)
    """

    return (str(entry["review_id"]),    # 0
            str(entry["user_id"]),      # 1
            str(entry["business_id"]),  # 2
            int(entry["stars"]),        # 3
            int(entry["useful"]),       # 4
            int(entry["funny"]),        # 5
            int(entry["cool"]),         # 6
            str(entry["text"]))         # 7


review_RDD = raw_review_RDD.map(get_review_tuple)

review_count = review_RDD.count()

print(f'There are {review_count} reviews in the dataset')
print(f'Reviews: {review_RDD.first()}')


There are 12205 reviews in the dataset
Reviews: ('4euQ70qJhaHbkG_iiAEDiA', 'A9Q-_QpJy1mHjlwwP2RwJg', 'EtKSTHV5Qx_Q7Aur9o4kQQ', 5, 0, 0, 0, 'If you want The Best burger in Philly, this is the right place to come. The bread was fresh, meat was nothing but juicy, service was a amiable, price was descent and affordable to everybody!')


In [20]:
def get_user_tuple(entry):
    """ Parse a row in the user dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the user dataset in JSON format
    Returns:
        tuple: (user_id, name, review_count, average_stars, useful, funny, cool, fans)
    """

    return (str(entry["user_id"]),          # 0
            str(entry["name"]),             # 1
            int(entry["review_count"]),     # 2
            float(entry["average_stars"]),  # 3
            int(entry["useful"]),           # 4
            int(entry["funny"]),            # 5
            int(entry["cool"]),             # 6
            int(entry["fans"]))             # 7


user_RDD = raw_user_RDD.map(get_user_tuple)

user_count = user_RDD.count()

print(f'There are {user_count} users in the dataset')
print(f'Users: {user_RDD.first()}')


There are 3292 users in the dataset
Users: ('QWxW_pApmpY9iPApC8P-UQ', 'Kelvin', 1306, 3.6, 3110, 1614, 1714, 80)


In [21]:
def get_business_tuple(entry):
    """ Parse a row in the business dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the business dataset in JSON format
    Returns:
        tuple: (business_id, name, city, state, stars, review_count, categories)
    """

    categories = [] if entry["categories"] == None \
                    else str(entry["categories"]).split(", ")
    
    return (str(entry["business_id"]),  # 0
            str(entry["name"]),         # 1
            str(entry["city"]),         # 2
            str(entry["state"]),        # 3
            float(entry["stars"]),      # 4
            int(entry["review_count"]), # 5
            categories)                 # 6

#TODO Attributes?

business_RDD = raw_business_RDD.map(get_business_tuple)

business_count = business_RDD.count()
# print(business_RDD.filter(lambda x: x[0]=='1H8ReY5GlGcHJz7umVidkg').first())

print(f'There are {business_count} business in the dataset')
print(f'Business: {business_RDD.first()}')

There are 150346 business in the dataset
Business: ('Pns2l4eNsfO8kk83dixA6A', 'Abby Rappoport, LAC, CMQ', 'Santa Barbara', 'CA', 5.0, 7, ['Doctors', 'Traditional Chinese Medicine', 'Naturopathic/Holistic', 'Acupuncture', 'Health & Medical', 'Nutritionists'])


# Calculate overall
### - for each business_id:

$$
  \Delta = \dfrac{1}{2} 
              \left( 
                  \dfrac{\text{useful} + \dfrac{1}{2}(\text{funny} + \text{cool})} 
                        {\text{best\ useful} + \dfrac{1}{2}(\text{best\ funny} + \text{best\ cool})}
                        + 
                  \dfrac{\text{fans}}
                        {\text{best\ fans}}
              \right)
$$
$$
\Delta : [0, 1]
$$
$$
  \text{overall} = \begin{cases} 
              \text{stars} + \Delta & \text{if stars } \ge 3\\ 
              \text{stars} - \Delta & \text{if stars } \lt 3
            \end{cases}
$$

In [22]:
# TODO fix all comment

#review_RDD.filter(lambda x: x[3]==0).count()


#stars - useful - funny - cool - numero fan


#if stars <3
    #stars - {[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2] + (numero fan/best user fans)}/2
#else
    #stars + {[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2] + (numero fan/best user fans)}/2

    #dati mancanti: numero fan; best fan; best useful/funny/cool per ristorante
#rdd_test_ufc = (id ristorsante, best useful, best funny, best cool)
#best fan = query su dataset utenti --> user_RDD.max().first()

#review_id - user_id - id_rist - overall


In [23]:
# get_overall_rating: function that generates the bias in order to incentivate or decrease the importance or overall rating of a review

# TODO fix function

#+ (fans/best_user_fans)) / 2

# def get_overall_rating(stars, best_useful, best_funny, best_cool, useful, funny, cool):
#     overall = (useful + (funny + cool)/2) / (best_useful + (best_funny + best_cool)/2) 
#     if stars < 3:
#         return stars - overall
#     else:
#         return stars + overall


In [24]:
def get_business_id_useful_funny_cool(row):
    """
        Writes a tuple with key business_id and attributes useful, funny, cool
    """
    return (row[2], (row[4], row[5], row[6]))

def get_max_useful_funny_cool(row1, row2):
    """
        Returns a new RDD with, for each business_id key, the maxima values of useful, funny, cool
    """
    return tuple(max(row1[i], row2[i]) for i in range(3))

review_tuple_RDD = review_RDD.map(get_business_id_useful_funny_cool)
review_best_ufc_RDD = review_tuple_RDD.reduceByKey(get_max_useful_funny_cool)

review_best_ufc_RDD.take(5)

# review_best_ufc_RDD tuple: (business_id, (useful, funny, cool)): for each reastaurant, this tuple takes the maxima values of useful, funny and cool. 


[('ymoEdJB5NE56sWw74uyX7A', (0, 0, 0)),
 ('q77zfw6P5CfG4n79QsOpBw', (4, 0, 0)),
 ('VenhDgh9fRrh9c0Xq0LacQ', (6, 2, 3)),
 ('A2q7d-CBM2-81tVkmS4JMw', (1, 3, 0)),
 ('US2-dEVvnOdde7Syi18TTw', (0, 0, 0))]

In [25]:
def rearrange_review_and_best_ufc_RDD(row):
    """
        Merge, for each business_id, the row values with the best useful, funny, cool ones
    """
    return (row[2], (row[0], row[1], row[3], row[4], row[5], row[6]))

review_ufc_RDD = review_RDD.map(rearrange_review_and_best_ufc_RDD).join(review_best_ufc_RDD)
review_ufc_RDD.first()

# (id_business, ((id_review, id_user, star, useful, funny , cool), (best useful, best funny, best cool)))


('ymoEdJB5NE56sWw74uyX7A',
 (('AghDcoBcndNPLLlkfi4jWg', 'ez5mEtjSbfb0nOtQiM4FCA', 5, 0, 0, 0), (0, 0, 0)))

In [26]:
def partial_overall(x):
    """
        Rearrange review_ufc_RDD and calculates the partial overall of the delta above
        partial_overall ={[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2]
    """
    key = (x[1][0][0], x[1][0][1], x[0], x[1][0][2])
    num = (x[1][0][3] + (x[1][0][4] + x[1][0][5])/2)
    den = (x[1][1][0] + (x[1][1][1] + x[1][1][2])/2)
    return (*key,(num/den if sum(x[1][1]) != 0 else 0))


partial_review_overall_RDD = review_ufc_RDD.map(partial_overall)
partial_review_overall_RDD.first()

# tuple: (review_id, user_id, buisness_id, stars, partial overall) 

('AghDcoBcndNPLLlkfi4jWg',
 'ez5mEtjSbfb0nOtQiM4FCA',
 'ymoEdJB5NE56sWw74uyX7A',
 5,
 0)

In [27]:
# tuple: (user_id, fans)
user_fans_RDD = user_RDD.map(lambda x: (x[0], x[7]))

# TODO add map to turn the result in a simple tuple
# tuple: (user_id, (review_id, buisness_id, stars, partial overall), fans)
partial_overall_fans_RDD = partial_review_overall_RDD.map(lambda x: (x[1], (x[0], x[2], x[3], x[4]))).join(user_fans_RDD)

partial_overall_fans_RDD.first()


('Eep1pCr1zMqub16tYgUFgQ',
 (('juj0IUHo7lkqtqqKHv-nMA', 'uSIGgRScZmpWlEcHtoaWlA', 5, 0), 1))

In [28]:
# user with the highest number of fans
best_user_fans = user_RDD.max(lambda x: x[7])
print("User with the highest number of fans: ",best_user_fans)

# TODO if best user fans = 0

def review_overall(x):
    user_id, review_id, business_id, stars= x[0], x[1][0][0], x[1][0][1], x[1][0][2]
    value = stars + (x[1][0][3]+x[1][1]/best_user_fans[7])/(2 if stars >=3 else -2)
    return (review_id, user_id, business_id, value)

    #tuple: (review_id, user_id, business_id, stars + (partial overall + fans/best_fans))


review_overall_RDD = partial_overall_fans_RDD.map(review_overall)

review_overall_RDD.sortBy(lambda x:x[3], ascending=False).take(5)


User with the highest number of fans:  ('pQ6jXAENt9gUHqQvXl4DTg', 'Faye', 1596, 3.73, 6126, 2540, 3793, 207)


[('hRujORdpS28APB9Guybj-Q',
  'pQ6jXAENt9gUHqQvXl4DTg',
  '_T8tOBDr3NKNxd0yl5GwjA',
  6.0),
 ('q47nNWcZkbZgj3U5yneaAQ',
  'Um5bfs5DH6eizgjH3xZsvg',
  'ocNvlc_zH3VdBbRpB6kwoA',
  5.990338164251208),
 ('q47nNWcZkbZgj3U5yneaAQ',
  'Um5bfs5DH6eizgjH3xZsvg',
  'ocNvlc_zH3VdBbRpB6kwoA',
  5.990338164251208),
 ('D931QjYSukjtRnN8dXopuQ',
  'Um5bfs5DH6eizgjH3xZsvg',
  'K7vvEwRd6Y8gRftjeuASuQ',
  5.990338164251208),
 ('D931QjYSukjtRnN8dXopuQ',
  'Um5bfs5DH6eizgjH3xZsvg',
  'K7vvEwRd6Y8gRftjeuASuQ',
  5.990338164251208)]

# Basic Recommendation

In [29]:
#Keeping businesses that have at least 20 reviews

#review_by_business_filtered_RDD = review_overall_RDD.map(lambda x: (x[2], (x[0], x[1], x[3]))).groupByKey()\
#                                                   .mapValues(list).filter(lambda x: len(x[1])>=20)
# review_by_business_filtered_RDD.first()


In [30]:
def sortFunction(tuple):
    """ Construct the sort string (does not perform actual sorting)
    Args:
        tuple: (rating, MovieName)
    Returns:
        sortString: the value to sort with, 'rating MovieName'
    """
    #key = unicode('%.3f' % tuple[0])
    if (tuple[1][1]!= None):
        value = '{:.3f}'.format(tuple[1][1])
    else:
        value = ''
    key = tuple[0]
    return (value + ' ' + key)


In [31]:
#Make, for each business, the average of the stars for all the reviews

# business_overall_RDD = review_by_business_filtered_RDD.map(lambda x: (x[0],  sum(i[2] for i in x[1])/len(x[1])))
# business_overall_RDD.first()

In [32]:
# top recommended businesses
#business_RDD.map(lambda x: (x[0], (x[1], x[6]))).leftOuterJoin(business_overall_RDD).sortBy(sortFunction, False).take(10)  #x[2], x[3],.filter(lambda x: 'Restaurants' in x[1][1])


# Collaborative Filtering

In [33]:
#TODO in the collaborative prt, take into consideration the categories and the State
#TODO based on the number of reviews, multiply by the percentage of the reviews made in a particular state (same for the categories)
# print(dict(sorted(business_RDD.flatMap(lambda x: tuple(x[6])).countByValue().items(), key=lambda x: x[1], reverse=True))) lista delle categorie più frequenti


## Preprocessing

In [34]:
# Preprocessing
# Because the differences in the quality of items and the rating scales of users are
# such important factors in determining the missing elements of the matrix M , it
# is often useful to remove these influences before doing anything else. The idea
# was introduced in Section 9.3.1. We can subtract from each nonblank element
# mij the average rating of user i. Then, the resulting matrix can be modified
# by subtracting the average rating (in the modified matrix) of item j. It is also
# possible to first subtract the average rating of item j and then subtract the
# average rating of user i in the modified matrix. The results one obtains from
# doing things in these two different orders need not be the same, but will tend
# to be close. A third option is to normalize by subtracting from mij the average
# of the average rating of user i and item j, that is, subtracting one half the sum
# of the user average and the item average.
# If we choose to normalize M , then when we make predictions, we need to
# undo the normalization. That is, if whatever prediction method we use results
# in estimate e for an element mij of the normalized matrix, then the value
# we predict for mij in the true utility matrix is e plus whatever amount was
# subtracted from row i and from column j during the normalization process.



#We have chosen to implement the third option
#review_overall_RDD.flatMap(lambda x : ((x[1], x[2]), 1)).reduceByKey(lambda x, y : x + y).sortBy(lambda x: x[1], False)


In [35]:
#media voti utente
#average_user_RDD = review_overall_RDD.map(lambda x: (x[1], x[3])).reduceByKey(lambda x,y: (x+y)/2)
# average_user_RDD.take(10)


In [36]:
#media voti business
#
#average_business_RDD = review_overall_RDD.map(lambda x: (x[2], x[3])).reduceByKey(lambda x,y: (x+y)/2)
# average_business_RDD.take(10)


In [37]:
#Merge review rates with average user
#partial_normalized_RDD = review_overall_RDD.map(lambda x: (x[1], (x[0], x[2], x[3]))).join(average_user_RDD).map(lambda x: (x[1][0][0], x[0], x[1][0][1], x[1][0][2], x[1][1]))
# partial_normalized_RDD.take(10)

# tuple: (review_id, user_id, buisness_id, overall, average_user)


In [38]:
# normalized_RDD = partial_normalized_RDD.map(lambda x: (x[1], (x[0], x[2], x[3]))) \
#                                        .join(average_business_RDD) \
#                                        .map(lambda x: ((x[1][0][0], x[0]), (x[1][0][1] - (x[1][1]+x[1][0][2])/2))) \
#                                        .reduceByKey(lambda x, y : (x+y)/2) \
#                                        .map(lambda x: (*x[0], x[1]))

# normalized_RDD.first()
# # tuple: (user_id, buisness_id, overall - normalized_value)


## Initialization

In [39]:
#We need to train and predict the vectors U and V
# train(trainingRDD, rank, iterations, regularization_parameter)  --> (U,V)
#We need to calculate RMSE between the utility and our matrix

#TODO initialize U and V with the overall average value

rank = 2
# regularization_parameter = 0.1

U_RDD = user_RDD.map(lambda x: (x[0], [1 for _ in range(rank)]))#.sortBy(lambda x: x[0])
V_RDD = business_RDD.map(lambda x: (x[0], [1 for _ in range(rank)]))#.sortBy(lambda x: x[0])

# U_RDD.take(5)
# V_RDD.take(5)


In [41]:
review_overall_bis = review_overall_RDD.map(lambda x: (x[1], x[2], x[3]))
#print(review_overall_RDD.filter(lambda x: x[1]=='cxuxXkcihfCbqt5Byrup8Q').first())

In [42]:
M_BC = sc.broadcast(review_overall_bis.collect())

# list(filter(lambda x: x[0]=='cxuxXkcihfCbqt5Byrup8Q', M_BC.value))


In [43]:
# todo no map
# V_BC = sc.broadcast(dict(V_RDD.map(lambda x : (x[0], x[1])).collect()))
V_BC = sc.broadcast(dict(V_RDD.collect()))
#V_BC.value['KsFuzQCOPhP8eHmt3rP11Q']



In [44]:
# todo no map
# U_BC = sc.broadcast(dict(U_RDD.map(lambda x : (x[0], x[1])).collect()))
U_BC = sc.broadcast(dict(U_RDD.collect()))

#len(U_BC.value)


## Performing Optimization

In [46]:
# normalized-->tuple: (user_id, buisness_id, overall - normalized_value)
# review_overall --> tuple: (review_id, user_id, business_id, stars + (partial overall + fans/best_fans))
#print(business_RDD.filter(lambda x: x[0]=='1H8ReY5GlGcHJz7umVidkg').first())

def update_U(entry):
    id = entry[0]
    U_value = entry[1]
    reviews = [i for i in M_BC.value if i[0] == id]
    for s in range(rank):
        res = 0
        den = 0
        for review in reviews:
            j = review[1] # business_id
            p = 0
            for k in [i for i in range(rank) if i != s]:
                #print(V_BC.value)
                p += U_value[k] * V_BC.value[j][k]

            res += V_BC.value[j][s] * (review[2] - p)
            den += V_BC.value[j][s]**2
        U_value[s] = res / (den if den > 0 else 1)

    return (id, U_value)

# entry = U_RDD.first()
# print(entry)
# id = entry[0]
# U_value = entry[1]
# reviews = [i for i in M_BC.value if i[0] == id]
# print(reviews)
# for s in range(2):
#     res = 0
#     den = 0
#     for review in reviews:
#         j = review[1] # business_id
#         print("j: ",j)
#         p = 0
#         for k in [i for i in range(rank) if i != s]:
#             #print(V_BC.value)
#             p += U_value[k] * V_BC.value[j][k]
#             print("p: ",p)
#         res += V_BC.value[j][s] * (review[2] - p)
#         print("res: ",res)
#         den += V_BC.value[j][s]**2
#         print("den: ", den)
#     U_value[s] = res / (den if den > 0 else 1)
#     print("U_value: ",U_value)
# U_RDD.map(lambda x :  update_U(x)).take(20)

#print(user_RDD.filter(lambda x: x[0]=='cxuxXkcihfCbqt5Byrup8Q').first()[2])


In [47]:
def update_V(entry):
    id = entry[0]
    V_value = entry[1]
    reviews = [i for i in M_BC.value if i[1] == id]

    # out = [1]*rank
    for s in range(rank):
        res = 0
        den = 0
        for review in reviews:
            j = review[0] # user_id
            p = 0
            for k in [i for i in range(rank) if i != s]:
                p += V_value[k] * U_BC.value[j][k]

            res += U_BC.value[j][s] * (review[2] - p)
            den += U_BC.value[j][s]**2
        V_value[s] = res / (den if den > 0 else 1)

    return (id, V_value)


# V_RDD.map(lambda x :  update_V(x)).take(20)

In [48]:
iterations = 1
for i in range(iterations):
    # V_BC = sc.broadcast(dict(V_RDD.map(lambda x : (x[0], x[1])).collect()))
    V_BC = sc.broadcast(dict(V_RDD.collect()))
    U_RDD = U_RDD.map(update_U)

    # U_BC = sc.broadcast(dict(U_RDD.map(lambda x : (x[0], x[1])).collect()))
    U_BC = sc.broadcast(dict(U_RDD.collect()))
    V_RDD = V_RDD.map(update_V)

U = U_RDD.collect()
V = V_RDD.collect()

In [49]:
U

[('QWxW_pApmpY9iPApC8P-UQ', [2.603112668330058, 1.0000000000000022]),
 ('dI8kmo3hMqY7xwV9TmJlqA', [2.9217825122915255, 1.0]),
 ('9G6hBwQZNAq9qZpJVhf0aA', [2.3055555555555554, 1.0]),
 ('YAIxImxlu3jlSdG91T2Pig', [3.4630023341092886, 0.9999999999999982]),
 ('iCos25g_O3Z5Am9K4383WA', [3.2984240603275587, 1.0000000000000018]),
 ('qxdDOcAOvtGqhHNfQLxBWA', [3.216494133885439, 0.9999999999999994]),
 ('pJJrq1CwBDJiZHuvYl4_7A', [3.1787439613526574, 0.9999999999999997]),
 ('IEc6gWVPXFxjixKHoDaLKg', [2.4813472893183035, 1.0000000000000004]),
 ('axrn0q3bVy0lcKBkby_NeA', [2.2222222222222223, 1.0]),
 ('la_u3YUkcixIqxPl3zmLNA', [3.0048309178743953, 1.0000000000000004]),
 ('8B2VyOnh9P1A_YNj1CeoDw', [3.2743628185907037, 1.0000000000000007]),
 ('RuqMLvQEYbPL06152nuLJw', [0.0, 0.0]),
 ('yx8wkvWdCH-eN6PHWKT7Wg', [4.115942028985507, 1.0000000000000004]),
 ('4YwesZEX8VPPJOKLfPD7lg', [2.4367461155785732, 1.0000000000000009]),
 ('rY2j_2ltp2sNn1XozueDVw', [4.0, 1.0]),
 ('wvbHhotj3PLTuTbHWnMOYg', [3.0, 1.0]),
 (

In [50]:
V

[('Pns2l4eNsfO8kk83dixA6A', [0.0, 0.0]),
 ('mpf3x-BjTdTEA3yCZrAYPw', [0.0, 0.0]),
 ('tUFrWirKiKi_TAnsVWINQQ', [0.0, 0.0]),
 ('MTSW4McQd7CbVtyjqoe9mw', [0.0, 0.0]),
 ('mWMc6_wTdE0EUBKIGXDVfA', [0.0, 0.0]),
 ('CF33F8-E6oudUQ46HnavjQ', [0.0, 0.0]),
 ('n_0UpQx1hsNbnPUSlodU8w', [0.0, 0.0]),
 ('qkRM_2X51Yqxk3btlwAQIg', [0.0, 0.0]),
 ('k0hlBqXX-Bt0vf1op7Jr1w', [0.0, 0.0]),
 ('bBDDEgkFA1Otx9Lfe7BZUQ', [0.0, 0.0]),
 ('UJsufbvfyfONHeWdvAHKjA', [1.1023807367805842, 0.9999999999999998]),
 ('eEOYSgkmpB90uNA7lDOMRA', [0.0, 0.0]),
 ('il_Ro8jwPlHresjw9EGmBg', [0.0, 0.0]),
 ('jaxMSoInw8Poo3XeMJt8lQ', [0.0, 0.0]),
 ('0bPLkL0QhhPO5kt1_EXmNQ', [0.0, 0.0]),
 ('MUTTqe8uqyMdBl186RmNeA', [0.0, 0.0]),
 ('rBmpy_Y1UbBx8ggHlyb7hA', [0.0, 0.0]),
 ('M0XSSHqrASOnhgbWDJIpQA', [0.0, 0.0]),
 ('8wGISYjYkE2tSqn3cDMu8A', [0.0, 0.0]),
 ('ROeacJQwBeh05Rqg7F6TCg', [0.0, 0.0]),
 ('WKMJwqnfZKsAae75RMP6jA', [0.0, 0.0]),
 ('qhDdDeI3K4jy2KyzwFN53w', [0.0, 0.0]),
 ('kfNv-JZpuN6TVNSO6hHdkw', [0.0, 0.0]),
 ('9OG5YkX1g2GReZM0AskizA',